<img src="imagenes/rn3.png" width="200">
<img src="http://www.identidadbuho.uson.mx/assets/letragrama-rgb-150.jpg" width="200">

# [Curso de Redes Neuronales](https://curso-redes-neuronales-unison.github.io/Temario/)

# Redes recurrentes, implementación simple

[**Julio Waissman Vilanova**](http://mat.uson.mx/~juliowaissman/), 9 de mayo de 2018.

En esta libreta vamos a explorar como desarrollar redes recurrentes, simples y basadas en unidades de memoria de largo y corto plazo (LSTM), aplicadas a la generación automática de texto.

Dado que estamos en México en año electoral, y que se vota por diputados y presidentes municipales en muchisimos municipios del país, nos preguntamos si podríamos inventar nuevos municipios para que todos los candidatos tuvieran un lugar que gobernar. Así, generamos una lista con el nombre de todos los municipios de México, y la vamos a usar para aprender los nombres, y generar nombres a partir de una red recurrente. Esto es interesante ya que en México hay muchos municipios cuyos nombres tienen raices del español, el nahuatl, direrentes lenguas mayas, varias lenguas de la familia yuto-azteca, e inclusive algunos que son palabras inventadas (como Mexicali). Así que generar nombres de municipios mexicanos *creibles* es un problema interesante.

El archivo con el nombre de los municipios se encuentra para su descarga [aqui](varios/municipios.txt).


## 1. Redes recurrentes: Desarrollar una red recurrente completamente a pie.

Con el fin de entender como funcionan las redes neuronales, vamos a aplicar un modelo de generaci´pn de texto *letra a letra*, en el cual tanto la arquitectura como el método de aprendizaje sean programados a mano. 

No vamos a pedir que lo programen todo solos, simplemente que utilicen el modelo programados en este [gist](https://gist.github.com/karpathy/d4dee566867f8291f086), y lo adapten a leer el nombre de los municipios y a generar nombres de municipios.

Para esto:

1. Copiar el contenido del *gist* y comentarlo en español (y cambiar algo de código de forma que quede mas claro para ti y para mi).

2. Copiar y comentar en español el contenido del método de verificción de gradiente (para limpiar el código de errores) y usarlo por unas cuantas iteraciones para demostrar que el algoritmo de entrenamiento funciona correctamente.

3. Ajustar los hiperparámetros del modelo, así como los parámetros del algoritmo de entrenamiento con el fin de generar una lista de nombres de municipios creibles, pero sin sobreaprendizaje (esto es, que copie vilmente el nombre de municipios existentes). 


In [1]:
import numpy as np

In [7]:
# data I/O
data = open('varios/municipios.txt', 'r',encoding="utf8").read() # should be simple plain text file
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print ('data has %d characters, %d unique.' % (data_size, vocab_size))
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }


data has 35530 characters, 68 unique.


In [8]:
# hyperparameters
hidden_size = 100 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 1e-1


In [9]:
# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias


In [16]:
def lossFun(inputs, targets, hprev):
  """
  inputs,targets are both list of integers.
  hprev is Hx1 array of initial hidden state
  returns the loss, gradients on model parameters, and last hidden state
  """
  xs, hs, ys, ps = {}, {}, {}, {}
  hs[-1] = np.copy(hprev)
  loss = 0
  # forward pass
  for t in range(len(inputs)):
    xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
    xs[t][inputs[t]] = 1
    hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
    ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
    loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
  # backward pass: compute gradients going backwards
  dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
  dbh, dby = np.zeros_like(bh), np.zeros_like(by)
  dhnext = np.zeros_like(hs[0])
  for t in reversed(range(len(inputs))):
    dy = np.copy(ps[t])
    dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
    dWhy += np.dot(dy, hs[t].T)
    dby += dy
    dh = np.dot(Why.T, dy) + dhnext # backprop into h
    dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
    dbh += dhraw
    dWxh += np.dot(dhraw, xs[t].T)
    dWhh += np.dot(dhraw, hs[t-1].T)
    dhnext = np.dot(Whh.T, dhraw)
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

In [17]:
def sample(h, seed_ix, n):
  """ 
  sample a sequence of integers from the model 
  h is memory state, seed_ix is seed letter for first time step
  """
  x = np.zeros((vocab_size, 1))
  x[seed_ix] = 1
  ixes = []
  for t in range(n):
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    y = np.dot(Why, h) + by
    p = np.exp(y) / np.sum(np.exp(y))
    ix = np.random.choice(range(vocab_size), p=p.ravel())
    x = np.zeros((vocab_size, 1))
    x[ix] = 1
    ixes.append(ix)
  return ixes


In [18]:
n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0
while True:
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  if p+seq_length+1 >= len(data) or n == 0: 
    hprev = np.zeros((hidden_size,1)) # reset RNN memory
    p = 0 # go from start of data
  inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
  targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

  # sample from the model now and then
  if n % 100 == 0:
    sample_ix = sample(hprev, inputs[0], 200)
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    print ('----\n %s \n----' % (txt, ))

  # forward seq_length characters through the net and fetch gradient
  loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001
  if n % 100 == 0: print  ('iter %d, loss: %f' % (n, smooth_loss)) # print progress
  
  # perform parameter update with Adagrad
  for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
    mem += dparam * dparam
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

  p += seq_length # move data pointer
  n += 1 # iteration counter 

----
 rLB.UBh,FlKYñ-0v6ñlKJd8xExBtblgg0áelbxUrs.TpA,ÁdtzEsdtJY0EKhxasoüvSQIPT.M,CGBRO680utIsXGXqVM0J00C6eRbMSÑvatüVí-ZbRfLúNAÑOIxVYOáXcVGhPÑüppz

BbsuBpqokñxLhrñP
K,Aútcoá0ytoi6Ccdtdi
dIDqLFt.lkqaz20SArsSúo 
----
iter 0, loss: 105.487694
----
 Vdeaaraacoauaraca yal  Fidl 
alaadaaad
oltazag
atouaalycdann

cmtá
aaaaAyzyda
eraLucamesad

a8aGcaa
alagxa 
avnT
raraóulasaradnrc neasidarircdadatzuot
ralcrnvn
cdcaAnad
rertzada
nraoa
ataAaraaaAály
Ar 
----
iter 100, loss: 105.615845
----
 te
CdetaáypaccelsstalhneoOoalaoGlka
hZhZeteládnlaacsotlte
CaoélC
Cobu orle
Cuoltla

o oalareéao
á uaráH itola ce
Coda
relcoaa
ChldogoCo oasa mlas t
áe aáxoFoyo CÁá o
Chate rgctsaloGeayájn emlcnau
cuC
 
----
iter 200, loss: 102.993156
----
 n
eral ddadlanee
EalAnedarao denar Fns
GeTroDuaa
Ben Geauhe
 ceretnntotieloreratandetrtocver
Fao
ndnelenaá
ríatlaáelahla
Filma Cía hdoabeTímennácne

Munarca
Cemerxner ndral.c Gel
Drorianeqeanresisalar 
----
iter 300, loss: 100.359420
----
 lla
Izala clconparaxolaralotalrtala 

----
 an Ascongus . Pereré
Juitétechuinyo
Ripa Amírú
SangRas
Pama Pía Risne
Sanda de Erúnc
Sochistla de Phuertreratohuonpin Elungén dírébeso Rám
San
Sonchiníaytan Rarína
Salcaz
San
Santínz
Necotin
Hténalanc 
----
iter 3500, loss: 55.199992
----
 n Lroscafinie Xandiol
San In
San Viscicociincám
San o Jebralcor
Papo
San Balidevas
San Fría
San Vallán Aesco Lomon Aliide PFzotlpal
Dan fevópec
San Arónvapapa
San Mcaquolo Juapoti
Sen Lanolocakgosiolo 
----
iter 3600, loss: 54.363780
----
  Nuidlos
San Martlar Ran
Panaman Cuo Jurcochuh
San Lan
Mureta Zat Sán Fraz Telcilacla
San Mca Alircastio Pergin Ponzso Elogucol Rara
San Miaticas
San Laz Piszol
San Manto Caquiala
Sandoa
San Marlaria
 
----
iter 3700, loss: 53.050341
----
 
San Pecalcma
San ódes
Santalán Cotín Juavaltapasguastolo
Satúo
San Macobo yquitlá
San Pellan San Ocox
Sin Titzablan
San Payaleo
San Tezatilvo
San yau
San PeYamica
San Vedra Queecángo Jíet
Telaxchixca 
----
iter 3800, loss: 51.707039
----
  Maríritlcá
Santana ía Acuitecápa


----
  Corúra
Temcuhua
Tlanal
Tuciyepa
Túrito
Toquiromico Tompepcan
Tallaza
Tuxcata
Tuñanton
Tlamandeu
Totdalan
Tenanga
Tiloac Amál epatla
Tjenbo
Tilatlán
Timucoca de LorHí Tidzamá
Qoxapa
Tozinchilpinguo
Tu 
----
iter 7000, loss: 45.345229
----
 oa PuezeutlHio
Zatliltepín
Yántepal Taparile E
Jénlalco
Jahhirti
Nadoconde Eritepao
Zochizcagolo
Tagaltendoc
Cortepec
Uamoalángo
LEnayico
Vigucañotlán Zoblano
Vilpatlán Zinango
Zonitlana
Zilligetintep 
----
iter 7100, loss: 45.815106
----
 o
Atquezmateáde Apunlepán
Jurso
Amayalco
Xixotayo
Zutmitlán Atolhtitlán del Réz
Opeya
Apaónla
Amanto Hinatlángo
Aourgo Coroxtán Ahuixca
Apactepec
Saramos Abzalca
Amanotina
Atapacaco
Atlalcinule
Pozucí 
----
iter 7200, loss: 46.251600
----
 unco
Camol Cuatsico
Chinven
Cahre
Caminaro
Acameuetapa
Cuanido
Coanela Juetad
Chinmé
AthilVica
Chantla de Palabla
Comomán
Chiapajliq
Cramon de Cango Coamas
Cáres
Canahui
Angotlán
Cuamapamalolu
Cobtell 
----
iter 7300, loss: 46.757659
----
 a
Zoyaros ÁzoríP Miranago
Xorrero


----
 jepec
Ditós
Namatlán
Mandeloz
Nacarepe
Matehuico
Emitlán dalgo
Minidros
Nagadezo
EntYapatlo
Horeo Altestimper
Nuaramolto enatetlán de El Arigudo
Merfemabo del Marco
Getedel Jagotdé
L. de Malrara Juicá 
----
iter 10500, loss: 47.363225
----
 ltinca Zala Ro
Vintla Aguecarta
Nujimín de Grerí
Luchovesta
Lavves
Santlo
Zachetitlán de Manca ñal Rínaquecuttey Pcha de Ros Raster
Siatis
Santzan la Baragos Lagquiltenchina An Bsa Okpo
Oxcotuchul uán 
----
iter 10600, loss: 48.023477
----
 s Tlaapa de Huerito de Corileri
San Guindes Nuaris
Sanuz
Sanito Yaddatis
Sinés Senocas
San Fesú Oéchen
San Cartón
Sana Manicincil
San
San Jalarus Silapan Santepa
San Sacaac
Guada
San In Crirla
San -al 
----
iter 10700, loss: 47.445611
----
  Loninguón
Rín Juanco Axtapac
San Liscolin Osqua
Sol PedesUs
Sen nMineL
Son Áastitlos
San Macilus Castír
San Monécos Platiaclaruapaz
San Míarten Mitatacacoro Ixtlama
San Astllo Coacaquap
San Manguas d 
----
iter 10800, loss: 46.385197
----
 o Tejapen
San Pedelams
San Ped

----
 relcolata
Santa Elas
Santiago Atenilan
Santiago Noroteata
Santiango
Santiag
San
Santa ónde Drre
Santiago Hoatingo
Sinta Crría TTltuipultipengo
Santiogo Tlóalicotepem
Santiaghé Tanatac
Santiago Zasotza 
----
iter 13900, loss: 42.441426
----
 gadan
Temango
Telueco
Tenemáno
Teltileo
Tixqué
Alcotepella
Temotuilla
Ticucapa
Zutzénconco
Telapan
Tepengo Testía
Teñojiné
Tatlalpade
Rebami
Zaxba
Textil
Tedahirpo de Tlaxintlán
Tecucápac
Telántiápec  
----
iter 14000, loss: 42.568637
----
 oncinepapa dr Cañe
Sonalpa
Tuitlán
TurónVindemal
Tramápalcat
Coreruán
Tigepo
Tuzal Tónelahulas
Tzancaro Teacolahilo
Tenatla
Toamolizto
Torango
Tlanixcul
Teutla
Tatalchalán
Tlamáro
Tyanchealc
Tixaulón
 
----
iter 14100, loss: 42.663365
----
 a Roroneder Eberi
Villa
Hicotepencile de szas
Vilaba
Juxzapan
Yabi
Viala
Gosiltaco
Tzttatza
Tanilo
Zinna Emo Imauzquizatla
ZacuchiQutipaca
Bapatepec
Zacatlán
Naquialpen
Zabanoca
Zanlanalonat
Zancoru
Z 
----
iter 14200, loss: 43.155619
----
 temác
Atantopatos
Calerote
Ach

...

----
 n del Miguel Ardena
Lafres
Lus Reyas
Zamata
Jacayo de Asavo
Gen Escartepec de Norialo
Herola Metetlamirta
Luíroztin de Domina Amito
Comconciatzcolul
Acapiltepec
Sanava Bosriedú
Mitipec
La Blanila Nach 
----
iter 598700, loss: 31.626091
----
 arlas
Bapecotepec
Nahuatlán
Mirotepes Alultica de Barzo
Loseño de Bos ente
Huitolosquiluco
Maguas Amitlatla
Santa Maríaflo Honache
Joquínco Tekolquita
Jiánca
Mortado
Flismé Lachi
Dos
Gostalton
Nuianda 
----
iter 598800, loss: 32.070613
----
 dras Istapecosi
Macías
San Eloscoro
Revista
Juárez
Naque Coavado
Santiago Hidalguas Vallanc Zaros Cuetepec
Mazapa
San Pella Ixquá
Puela
Adella Pupotla
Santiago Minas
Parro
San Juario
Mardalas
Sueyeon
 
----
iter 598900, loss: 32.499405
----
 Bes Ampica
San Jorede
Lariso Zepacán
Pintlaspa Imatavaratlán
San Francisco Huepa
Lega de Carán
San Francis
San Galon del An Botambargo
San Mitlán Hucupán
San FlmCa Indalegria
San Mito de la Meredado
P 
----
iter 599000, loss: 31.824385
----
 NTanan
San Maguel l Carsa 

----
 hilapicú
Tosma Camoltepec
Santa Coniala
Santa Anta
Saniunu del Maza
Santo Avengo
Sondim
Salta Matitlán
Santa María Xoteitlán
Santa Lacavo de Ichimé
Suemaxtitlán
Santa María Martízan
Sancia
Goanitalo
S 
----
iter 602100, loss: 29.515945
----
 n
Santiago Imuejoco
Santab
Gugus
Prigaste
Santiago Techucayán
Conatlapag
Santo Dilla
Nogalepac
Xinac
Santiago Chattitlán
Santiago Camatitlapa
Santa Lagual Lagunmo
Santa Ma Rende
Santicod Luzalmen
Sant 
----
iter 602200, loss: 28.601434
----
 
Tuchiteltepec de Estonanto
Tezolá
Tuxtea
Santo Roso de Vedo
Santialo de Pañillo
Santo Domilgo Nuchilauco
Teotepan de Aotopeca
Tematlánc
Tenallo
Santo Tomásto
Santo Xo Sicopimoyán
Tillatla
Tuxcuhuar
S 
----
iter 602300, loss: 29.020809
----
 lachuatlá
Tonotlántlán
Tempelascaa
Acaneti de Jolapam
Tloxcan
Tlila
Tasloquilla
Santos Fronde
Tongo
Tarampaholco
Tuxtachaltepec
Vilá
Cmabatlán
Tucuetlán
Pilantebez Singo Jeyálcago
Villa Solco
Ahuatlán 
----
iter 602400, loss: 29.358886
----
 copana
Alahuaatit
Ayotitlá

----
 yco
Coyahualco
Chiquino Blan ebemo
Atenalhi
Cangamara
Villa
Chidalultzim
Buctoaluco
Tercho
Alelul Oscabigria
Eschoreti
Chiatzotla de Hidalgo
Capacuaca
Candelanco
Chelpanangón
Ateatic
Frisrmán
Guadalun 
----
iter 605500, loss: 30.473214
----
 o
Catatzarcabin
Elupa Ñusangó
Cayuconco
Coamulapan
Comima
Chénco
Chulcoyo de Cumdo Teñamanca
Chileutlaco
Coapilchelomonega
Cilantinguaul
Cocholcoyo
Cozuma
Raíz
Atestinta Jironen do Guerres
Buerralirad 
----
iter 605600, loss: 30.845011
----
  Oelchitla Vilos
Huapan
El Coscalula
Juróndezo de Mixco de las Chiavós
Gran
Igoquitoxú

Luando
Guarreyos Réjóro
Monmanispa del Marsienigo Altonio Jesúc
Chueyan
Huicontepec bal Ramas
Miaremara
San Mara 
----
iter 605700, loss: 31.221357
----
 nde
Tenaticoalucoquitlán de Guerécerato
Jomhláe
Heromo Tepe
Galertora del Plortepec
La Matoatza
Pihuilcha
Juchiatza
Santa Miría Tarotíz
Hidola
Juacuatlahuaco
Hejucapula
Higraz
Rerza
La Terterria
Ixten 
----
iter 605800, loss: 31.430025
----
 epec -D Olas
La Miderto
Me

----
 n Guanvo
San Berosiopal
San Juan Tuavinte
Serío Villa Jalapanco
San Anisteaca
Pedallindo
Peñúpesca
San Pedro Tinel Cmotes
Quilmarapan
Palojios
San Berosol de la Arine lla Olo
Solo de Ao de Frala
Suldr 
----
iter 608900, loss: 32.049904
----
 26 - Zatinco
Nuejim
San Josés Josés de Istín Jarteaca
San Ciagón Huerro Alméno
San Juárez
San Jusúver Cabras Atima
San Felintíc
Lánjúa
San Igutasquio de Mereras
Pánnde
San Felipe Techacalua
San Crunes 
----
iter 609000, loss: 31.142598
----
 i
San Miguel Teteniltiro Temelcorigostañoz Alalos
San Miguel Tetetlán
San Pedré Arezo de Conanito
San Miguel Calelis
Sanci San Marcío El e Toriscos
San Antori
Saltia de la Ramingo
Pulatem
San Antonio  
----
iter 609100, loss: 30.290064
----
 a Marua de Hitlatlán
Santiago Téchiltipan
Santa Caun Apeso
San Pedro Tehuetlán
Santiago Huacaronco
Santa María Teteltepec
San Tedeo Santa Quiero
Santa Bal Pecekoti
Pxtechosote
Santiago Cazalan
Santa M 
----
iter 609200, loss: 29.476351
----
 ar
Santiago Yaltaro
Santia

KeyboardInterrupt: 

In [ ]:
219800 iter

In [ ]:
# gradient checking
from random import uniform
def gradCheck(inputs, target, hprev):
  global Wxh, Whh, Why, bh, by
  num_checks, delta = 10, 1e-5
  _, dWxh, dWhh, dWhy, dbh, dby, _ = lossFun(inputs, targets, hprev)
  for param,dparam,name in zip([Wxh, Whh, Why, bh, by], [dWxh, dWhh, dWhy, dbh, dby], ['Wxh', 'Whh', 'Why', 'bh', 'by']):
    s0 = dparam.shape
    s1 = param.shape
    assert s0 == s1, 'Error dims dont match: %s and %s.' % (`s0`, `s1`)
    print name
    for i in range(num_checks):
      ri = int(uniform(0,param.size))
      # evaluate cost at [x + delta] and [x - delta]
      old_val = param.flat[ri]
      param.flat[ri] = old_val + delta
      cg0, _, _, _, _, _, _ = lossFun(inputs, targets, hprev)
      param.flat[ri] = old_val - delta
      cg1, _, _, _, _, _, _ = lossFun(inputs, targets, hprev)
      param.flat[ri] = old_val # reset old value for this parameter
      # fetch both numerical and analytic gradient
      grad_analytic = dparam.flat[ri]
      grad_numerical = (cg0 - cg1) / ( 2 * delta )
      rel_error = abs(grad_analytic - grad_numerical) / abs(grad_numerical + grad_analytic)
      print '%f, %f => %e ' % (grad_numerical, grad_analytic, rel_error)
      # rel_error should be on order of 1e-7 or less

## Redes recurrentes tipo LSTM

Las redes con unidades LSTM las vimos platicadas en clase, pero no hay nada mejor para entender un tema que implementarlo y comparar los resultados con la red recurrente simple, sin memoria, para esto vamos a hacer lo mismo que antes, pero con unidades LSTM.

Para esto vamos a utilizar otro [gist](https://gist.github.com/karpathy/587454dc0146a6ae21fc) del mismo autor (que es una referencia obligada en el tema, por cierto). En este *gist*, el autor presenta un modelo de redes recurrentes LSTM desarrollado con *numpy* incluido el método de entrenamiento, pero no lo aplica a el modelado *letra a letra* como el gist pasado. Para esta parte de la libreta, lo que tienen que realizar es lo siguiente:


1. Copiar el contenido del *gist* y comentarlo en español (y cambiar algo de código de forma que quede mñas claro para ti y para mi).

2. Adaptar el modelo propuesto para usarlo en la generación de nombres de municipios.

3. Ajustar los hiperparámetros del modelo, así como los parámetros del algoritmo de entrenamiento con el fin de generar una lista de nombres de municipios creibles, pero sin sobreaprendizaje (esto es, que copie vilmente el nombre de municipios existentes). 


In [1]:
import numpy as np
import code

In [20]:
class LSTM:
  
  @staticmethod
  def init(input_size, hidden_size, fancy_forget_bias_init = 3):
    """ 
    Initialize parameters of the LSTM (both weights and biases in one matrix) 
    One might way to have a positive fancy_forget_bias_init number (e.g. maybe even up to 5, in some papers)
    """
    # +1 for the biases, which will be the first row of WLSTM
    WLSTM = np.random.randn(input_size + hidden_size + 1, 4 * hidden_size) / np.sqrt(input_size + hidden_size)
    WLSTM[0,:] = 0 # initialize biases to zero
    if fancy_forget_bias_init != 0:
      # forget gates get little bit negative bias initially to encourage them to be turned off
      # remember that due to Xavier initialization above, the raw output activations from gates before
      # nonlinearity are zero mean and on order of standard deviation ~1
      WLSTM[0,hidden_size:2*hidden_size] = fancy_forget_bias_init
    return WLSTM
  
  @staticmethod
  def forward(X, WLSTM, c0 = None, h0 = None):
    """
    X should be of shape (n,b,input_size), where n = length of sequence, b = batch size
    """
    n,b,input_size = X.shape
    d = int(WLSTM.shape[1]/4) # hidden size
    if c0 is None: c0 = np.zeros((b,d))
    if h0 is None: h0 = np.zeros((b,d))
    
    # Perform the LSTM forward pass with X as the input
    xphpb = WLSTM.shape[0] # x plus h plus bias, lol
    Hin = np.zeros((n, b, xphpb)) # input [1, xt, ht-1] to each tick of the LSTM
    Hout = np.zeros((n, b, d)) # hidden representation of the LSTM (gated cell content)
    IFOG = np.zeros((n, b, d * 4)) # input, forget, output, gate (IFOG)
    IFOGf = np.zeros((n, b, d * 4)) # after nonlinearity
    C = np.zeros((n, b, d)) # cell content
    Ct = np.zeros((n, b, d)) # tanh of cell content
    for t in range(n):
      # concat [x,h] as input to the LSTM
      prevh = Hout[t-1] if t > 0 else h0
      Hin[t,:,0] = 1 # bias
      Hin[t,:,1:input_size+1] = X[t]
      Hin[t,:,input_size+1:] = prevh
      # compute all gate activations. dots: (most work is this line)
      IFOG[t] = Hin[t].dot(WLSTM)
      # non-linearities
      IFOGf[t,:,:3*d] = 1.0/(1.0+np.exp(-IFOG[t,:,:3*d])) # sigmoids; these are the gates
      IFOGf[t,:,3*d:] = np.tanh(IFOG[t,:,3*d:]) # tanh
      # compute the cell activation
      prevc = C[t-1] if t > 0 else c0
      C[t] = IFOGf[t,:,:d] * IFOGf[t,:,3*d:] + IFOGf[t,:,d:2*d] * prevc
      Ct[t] = np.tanh(C[t])
      Hout[t] = IFOGf[t,:,2*d:3*d] * Ct[t]

    cache = {}
    cache['WLSTM'] = WLSTM
    cache['Hout'] = Hout
    cache['IFOGf'] = IFOGf
    cache['IFOG'] = IFOG
    cache['C'] = C
    cache['Ct'] = Ct
    cache['Hin'] = Hin
    cache['c0'] = c0
    cache['h0'] = h0

    # return C[t], as well so we can continue LSTM with prev state init if needed
    return Hout, C[t], Hout[t], cache
  
  @staticmethod
  def backward(dHout_in, cache, dcn = None, dhn = None): 

    WLSTM = cache['WLSTM']
    Hout = cache['Hout']
    IFOGf = cache['IFOGf']
    IFOG = cache['IFOG']
    C = cache['C']
    Ct = cache['Ct']
    Hin = cache['Hin']
    c0 = cache['c0']
    h0 = cache['h0']
    n,b,d = Hout.shape
    input_size = WLSTM.shape[0] - d - 1 # -1 due to bias
 
    # backprop the LSTM
    dIFOG = np.zeros(IFOG.shape)
    dIFOGf = np.zeros(IFOGf.shape)
    dWLSTM = np.zeros(WLSTM.shape)
    dHin = np.zeros(Hin.shape)
    dC = np.zeros(C.shape)
    dX = np.zeros((n,b,input_size))
    dh0 = np.zeros((b, d))
    dc0 = np.zeros((b, d))
    dHout = dHout_in.copy() # make a copy so we don't have any funny side effects
    if dcn is not None: dC[n-1] += dcn.copy() # carry over gradients from later
    if dhn is not None: dHout[n-1] += dhn.copy()
    for t in reversed(range(n)):
 
      tanhCt = Ct[t]
      dIFOGf[t,:,2*d:3*d] = tanhCt * dHout[t]
      # backprop tanh non-linearity first then continue backprop
      dC[t] += (1-tanhCt**2) * (IFOGf[t,:,2*d:3*d] * dHout[t])
 
      if t > 0:
        dIFOGf[t,:,d:2*d] = C[t-1] * dC[t]
        dC[t-1] += IFOGf[t,:,d:2*d] * dC[t]
      else:
        dIFOGf[t,:,d:2*d] = c0 * dC[t]
        dc0 = IFOGf[t,:,d:2*d] * dC[t]
      dIFOGf[t,:,:d] = IFOGf[t,:,3*d:] * dC[t]
      dIFOGf[t,:,3*d:] = IFOGf[t,:,:d] * dC[t]
      
      # backprop activation functions
      dIFOG[t,:,3*d:] = (1 - IFOGf[t,:,3*d:] ** 2) * dIFOGf[t,:,3*d:]
      y = IFOGf[t,:,:3*d]
      dIFOG[t,:,:3*d] = (y*(1.0-y)) * dIFOGf[t,:,:3*d]
 
      # backprop matrix multiply
      dWLSTM += np.dot(Hin[t].transpose(), dIFOG[t])
      dHin[t] = dIFOG[t].dot(WLSTM.transpose())
 
      # backprop the identity transforms into Hin
      dX[t] = dHin[t,:,1:input_size+1]
      if t > 0:
        dHout[t-1,:] += dHin[t,:,input_size+1:]
      else:
        dh0 += dHin[t,:,input_size+1:]
 
    return dX, dWLSTM, dc0, dh0

In [24]:
# -------------------
# TEST CASES
# -------------------

def checkSequentialMatchesBatch():
  """ check LSTM I/O forward/backward interactions """

  #n,b,d = (5, 3, 4) # sequence length, batch size, hidden size
  input_size = 10
  WLSTM = LSTM.init(input_size, d) # input size, hidden size
  #X = np.random.randn(n,b,input_size)
  h0 = np.random.randn(b,d)
  c0 = np.random.randn(b,d)

  # sequential forward
  cprev = c0
  hprev = h0
  caches = [{} for t in range(n)]
  Hcat = np.zeros((n,b,d))
  for t in range(n):
    xt = X[t:t+1]
    _, cprev, hprev, cache = LSTM.forward(xt, WLSTM, cprev, hprev)
    caches[t] = cache
    Hcat[t] = hprev

  # sanity check: perform batch forward to check that we get the same thing
  H, _, _, batch_cache = LSTM.forward(X, WLSTM, c0, h0)
  assert np.allclose(H, Hcat), 'Sequential and Batch forward don''t match!'

  # eval loss
  wrand = np.random.randn(*Hcat.shape)
  loss = np.sum(Hcat * wrand)
  dH = wrand

  # get the batched version gradients
  BdX, BdWLSTM, Bdc0, Bdh0 = LSTM.backward(dH, batch_cache)

  # now perform sequential backward
  dX = np.zeros_like(X)
  dWLSTM = np.zeros_like(WLSTM)
  dc0 = np.zeros_like(c0)
  dh0 = np.zeros_like(h0)
  dcnext = None
  dhnext = None
  for t in reversed(range(n)):
    dht = dH[t].reshape(1, b, d)
    dx, dWLSTMt, dcprev, dhprev = LSTM.backward(dht, caches[t], dcnext, dhnext)
    dhnext = dhprev
    dcnext = dcprev

    dWLSTM += dWLSTMt # accumulate LSTM gradient
    dX[t] = dx[0]
    if t == 0:
      dc0 = dcprev
      dh0 = dhprev

  # and make sure the gradients match
  print ('Making sure batched version agrees with sequential version: (should all be True)')
  print (np.allclose(BdX, dX))
  print (np.allclose(BdWLSTM, dWLSTM))
  print (np.allclose(Bdc0, dc0))
  print (np.allclose(Bdh0, dh0))
  

def checkBatchGradient():
  """ check that the batch gradient is correct """

  # lets gradient check this beast
  n,b,d = (5, 3, 4) # sequence length, batch size, hidden size
  input_size = 10
  WLSTM = LSTM.init(input_size, d) # input size, hidden size
  X = np.random.randn(n,b,input_size)
  h0 = np.random.randn(b,d)
  c0 = np.random.randn(b,d)

  # batch forward backward
  H, Ct, Ht, cache = LSTM.forward(X, WLSTM, c0, h0)
  wrand = np.random.randn(*H.shape)
  loss = np.sum(H * wrand) # weighted sum is a nice hash to use I think
  dH = wrand
  dX, dWLSTM, dc0, dh0 = LSTM.backward(dH, cache)

  def fwd():
    h,_,_,_ = LSTM.forward(X, WLSTM, c0, h0)
    return np.sum(h * wrand)

  # now gradient check all
  delta = 1e-5
  rel_error_thr_warning = 1e-2
  rel_error_thr_error = 1
  tocheck = [X, WLSTM, c0, h0]
  grads_analytic = [dX, dWLSTM, dc0, dh0]
  names = ['X', 'WLSTM', 'c0', 'h0']
  for j in range(len(tocheck)):
    mat = tocheck[j]
    dmat = grads_analytic[j]
    name = names[j]
    # gradcheck
    for i in range(mat.size):
      old_val = mat.flat[i]
      mat.flat[i] = old_val + delta
      loss0 = fwd()
      mat.flat[i] = old_val - delta
      loss1 = fwd()
      mat.flat[i] = old_val

      grad_analytic = dmat.flat[i]
      grad_numerical = (loss0 - loss1) / (2 * delta)

      if grad_numerical == 0 and grad_analytic == 0:
        rel_error = 0 # both are zero, OK.
        status = 'OK'
      elif abs(grad_numerical) < 1e-7 and abs(grad_analytic) < 1e-7:
        rel_error = 0 # not enough precision to check this
        status = 'VAL SMALL WARNING'
      else:
        rel_error = abs(grad_analytic - grad_numerical) / abs(grad_numerical + grad_analytic)
        status = 'OK'
        if rel_error > rel_error_thr_warning: status = 'WARNING'
        if rel_error > rel_error_thr_error: status = '!!!!! NOTOK'

      # print stats
      print ('{} checking param {} index {} (val = {}), analytic = {}, numerical = {}, relative error = {}'.format(status, name,np.unravel_index(i, mat.shape), old_val, grad_analytic, grad_numerical, rel_error))


if __name__ == "__main__":

  #checkSequentialMatchesBatch()
  #input('check OK, press key to continue to gradient check')
  #checkBatchGradient()
  #print ('every line should start with OK. Have a nice day!')
    
  # lets gradient check this beast
  n,b,d = (5, 3, 4) # sequence length, batch size, hidden size
  input_size = 10
  WLSTM = LSTM.init(input_size, d) # input size, hidden size
  X = np.random.randn(n,b,input_size)
  h0 = np.random.randn(b,d)
  c0 = np.random.randn(b,d)

  # batch forward backward
  H, Ct, Ht, cache = LSTM.forward(X, WLSTM, c0, h0)  



Making sure batched version agrees with sequential version: (should all be True)
True
True
True
True
check OK, press key to continue to gradient check
OK checking param X index (0, 0, 0) (val = 0.8697007376589942), analytic = 0.09996672959265124, numerical = 0.09996672960532747, relative error = 6.340226949744201e-11
OK checking param X index (0, 0, 1) (val = 0.989510969448005), analytic = -0.04899668674133324, numerical = -0.04899668675228951, relative error = 1.1180623977289539e-10
OK checking param X index (0, 0, 2) (val = -0.9170081261420228), analytic = 0.0795925490220265, numerical = 0.07959254912925928, relative error = 6.736357828585882e-10
OK checking param X index (0, 0, 3) (val = 0.04876504395898353), analytic = 0.22922501512841112, numerical = 0.22922501505817647, relative error = 1.5320023296649197e-10
OK checking param X index (0, 0, 4) (val = 0.6051025147615469), analytic = -0.07764464915087421, numerical = -0.07764464911019786, relative error = 2.6193917824157774e-10
OK

OK checking param WLSTM index (0, 0) (val = 0.0), analytic = -0.2803815837752009, numerical = -0.28038158372822863, relative error = 8.376487996623777e-11
OK checking param WLSTM index (0, 1) (val = 0.0), analytic = -0.3172965157762634, numerical = -0.3172965158171337, relative error = 6.440396459443844e-11
OK checking param WLSTM index (0, 2) (val = 0.0), analytic = 0.23010086063119622, numerical = 0.23010086054320308, relative error = 1.9120558768705523e-10
OK checking param WLSTM index (0, 3) (val = 0.0), analytic = -0.009809062333546087, numerical = -0.009809062362364784, relative error = 1.468983293517408e-09
OK checking param WLSTM index (0, 4) (val = 3.0), analytic = -0.05086686670138835, numerical = -0.050866866674326154, relative error = 2.660100230918426e-10
OK checking param WLSTM index (0, 5) (val = 3.0), analytic = -0.05270710982314543, numerical = -0.05270710987659299, relative error = 5.070241854556741e-10
OK checking param WLSTM index (0, 6) (val = 3.0), analytic = 0.13

OK checking param WLSTM index (9, 15) (val = -0.22920543061815493), analytic = -0.2515649604511783, numerical = -0.25156496050726673, relative error = 1.114789825778304e-10
OK checking param WLSTM index (10, 0) (val = -0.5703186923707607), analytic = -0.2386850408081071, numerical = -0.23868504079871397, relative error = 1.9676819994564502e-11
OK checking param WLSTM index (10, 1) (val = 0.11432066329211997), analytic = -0.13067350674606892, numerical = -0.13067350668194422, relative error = 2.453623047166464e-10
OK checking param WLSTM index (10, 2) (val = -0.23504536989031402), analytic = -0.0990458747284934, numerical = -0.09904587470188629, relative error = 1.3431708026364126e-10
OK checking param WLSTM index (10, 3) (val = 0.045432096356245025), analytic = -0.20868979212725336, numerical = -0.20868979206944968, relative error = 1.3849186855085886e-10
OK checking param WLSTM index (10, 4) (val = 0.3754031321111446), analytic = 0.048853036433561904, numerical = 0.048853036327400894,

Por último, agrega en esta misma celda comentarios sobre la comparación entre los dos modelos vistos, diferencias, similitudes, ventajas, desventajas. Recuerda es una opinión personal basada en tu trabajo, no pongas lo que dice la literatura si no lo que tu experimentaste a la hora de desarrollar y aplicar los modelos, así no concuerde con lo que leas en otros lados.